In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "emilyalsentzer/Bio_ClinicalBERT"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)


c:\Python\envs\action-learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python\envs\action-learning\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prade\.cache\huggingface\hub\models--emilyalsentzer--Bio_ClinicalBERT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate 

In [13]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


# Load the dataset
df = pd.read_csv('C:/Pradeepa/SEMESTER3_EPITA/Action Learning/medical_data.csv')

# Function to de-identify text
def de_identify(text):
    text = re.sub(r'\[\*\*.*?\*\*\]', '[REDACTED]', text)
    return text

# Apply de-identification to the TEXT column
df['TEXT'] = df['TEXT'].apply(de_identify)

# Adding a dummy label column for demonstration

df['LABEL'] = np.random.randint(0, 2, df.shape[0])

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(df['TEXT'].tolist(), df['LABEL'].tolist(), test_size=0.2, random_state=42)


In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

class MedicalDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MedicalDataset(train_texts, train_labels)
val_dataset = MedicalDataset(val_texts, val_labels)




In [15]:
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3, 
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

c:\Python\envs\action-learning\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [18]:
trainer.train()


IndexError: index out of range in self

In [2]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import numpy as np

# Load the dataset
df = pd.read_csv('C:/Pradeepa/SEMESTER3_EPITA/Action Learning/medical_data.csv')

# Function to de-identify text
def de_identify(text):
    text = re.sub(r'\[\*\*.*?\*\*\]', '[REDACTED]', text)
    return text

# Apply de-identification to the TEXT column
df['TEXT'] = df['TEXT'].apply(de_identify)

# Adding a dummy label column for demonstration
df['LABEL'] = np.random.randint(0, 2, df.shape[0])

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(df['TEXT'].tolist(), df['LABEL'].tolist(), test_size=0.2, random_state=42)

# Verify the unique labels
unique_train_labels = set(train_labels)
unique_val_labels = set(val_labels)
print(f"Unique training labels: {unique_train_labels}")
print(f"Unique validation labels: {unique_val_labels}")

# Ensure labels are within the correct range
num_labels = 2  # Adjust this based on your specific use case
assert all(label < num_labels for label in unique_train_labels)
assert all(label < num_labels for label in unique_val_labels)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

class MedicalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Create the Dataset
train_dataset = MedicalDataset(train_encodings, train_labels)
val_dataset = MedicalDataset(val_encodings, val_labels)

# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=8,    # batch size for training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Define compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Create Trainer instance
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # custom compute metrics function
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Unique training labels: {0, 1}
Unique validation labels: {0, 1}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Python\envs\action-learning\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(













                                                  
                                            
 33%|███▎      | 75/225 [56:13<53:05, 21.24s/it]

{'loss': 0.6872, 'grad_norm': 4.3213419914245605, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.13}














                                                  
                                            
 33%|███▎      | 75/225 [1:00:03<53:05, 21.24s/it]

{'loss': 0.7048, 'grad_norm': 4.270304203033447, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.27}


...

{'loss': 0.6866, 'grad_norm': 5.134488105773926, 'learning_rate': 2.2000000000000003e-05, 'epoch': 2.93}



























                                                  

                                             
                                              


 33%|███▎      | 75/225 [2:30:00<53:05, 21.24s/it]




                                                  

                                             
100%|██████████| 225/225 [1:37:59<00:00, 26.13s/it]


{'eval_loss': 0.7147508859634399, 'eval_accuracy': 0.42953020134228187, 'eval_precision': 0.4415454016997113, 'eval_recall': 0.42953020134228187, 'eval_f1': 0.3539742704467815, 'eval_runtime': 149.6799, 'eval_samples_per_second': 0.995, 'eval_steps_per_second': 0.067, 'epoch': 3.0}
{'train_runtime': 5879.6911, 'train_samples_per_second': 0.304, 'train_steps_per_second': 0.038, 'train_loss': 0.6977479733361138, 'epoch': 3.0}












100%|██████████| 10/10 [02:11<00:00, 13.16s/it]

{'eval_loss': 0.7147508859634399, 'eval_accuracy': 0.42953020134228187, 'eval_precision': 0.4415454016997113, 'eval_recall': 0.42953020134228187, 'eval_f1': 0.3539742704467815, 'eval_runtime': 146.367, 'eval_samples_per_second': 1.018, 'eval_steps_per_second': 0.068, 'epoch': 3.0}
